# Season Stat Tables 

## Imports

In [1]:
# !pip install nba_api 
# !pip install sqlalchemy 

# Base imports for data science 
import requests, numpy as np, pandas as pd 
pd.set_option('display.max_columns', 0)

# NBA API imports 
from nba_api.stats.endpoints import playercareerstats 
from nba_api.stats.static import players

# Database imports 
from sqlalchemy import create_engine
import sqlite3



## Functions

In [2]:
def get_stats(id, season, stats):
    # Find player's career stats table and season 
    season_stats = playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[0].tail(1)  
    # Find and return array of relevant statistics 
    new_stats = []
    for stat in stats:
        new_stats.append(np.sum(season_stats[stat])) 
        
    return new_stats 

# Takes a lot of time (~5-10 minutes) 
def get_table(season, stats):
    # Get table of all players in NBA history and select relevant variables 
    players_data = pd.json_normalize(players.get_players()) 
    players_data = (players_data.loc[players_data.is_active==True, ['id', 'full_name']]
                    .rename(columns={'id':'ID', 'full_name':'Name'})
                    .sort_values(by='ID')
                    .iloc[0:531, :]
    ) 
    # Create stats column by getting stats from each individual player 
    players_data['stats'] = players_data['ID'].apply(lambda x: get_stats(x, season, stats)) 
    players_data[stats] = pd.DataFrame(players_data['stats'].tolist(), index=players_data.index)
    players_data = (players_data
                    .drop('stats', axis=1)  
                    .set_index('ID') 
                    .rename(columns={'PLAYER_AGE':'Age'})
    )
    
    
    return players_data 

def add_pg_stats(df, pg_cols, old_cols):
    df[pg_cols] = np.round(df[old_cols].divide(df['GP'], axis=0), 1)  
    df.fillna(0, inplace=True)
    
    return df 

In [3]:
players_data = pd.json_normalize(players.get_players()) 
(players_data.loc[players_data.is_active==True, ['id', 'full_name']]
                    .rename(columns={'id':'ID', 'full_name':'Name'})
                    .sort_values(by='ID')
                    # .iloc[0:531, :]
    ) 

,ID,Name
2111,2544,LeBron James
3376,101108,Chris Paul
2635,200768,Kyle Lowry
4383,200782,P.J. Tucker
1171,201142,Kevin Durant
...,...,...
3486,1641854,Craig Porter
3593,1641871,Duop Reath
1056,1641926,Dexter Dennis
349,1641931,Onuralp Bitim


## Table Creation

	PLAYER_ID	SEASON_ID	LEAGUE_ID	TEAM_ID	TEAM_ABBREVIATION	PLAYER_AGE	GP	GS	MIN	FGM	FGA	FG_PCT	FG3M	FG3A	FG3_PCT	FTM	FTA	FT_PCT	OREB	DREB	REB	AST	STL	BLK	TOV	PF	PTS	

# --- Warning: 15+ mins to load --- 

In [4]:
season = '2023-24'
stats = ['PLAYER_AGE', 'MIN', 'GP','PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'FG_PCT', 'FG3_PCT', 'FT_PCT'] 
season_stats = get_table(season, stats) 
season_stats 

,Name,Age,MIN,GP,PTS,REB,AST,STL,BLK,TOV,PF,FG_PCT,FG3_PCT,FT_PCT
ID,,,,,,,,,,,,,,
2544,LeBron James,39.0,2504.0,71,1822,518,589,89,38,245,78,0.540,0.410,0.750
101108,Chris Paul,39.0,1531.0,58,533,225,393,70,6,76,108,0.441,0.371,0.827
200768,Kyle Lowry,38.0,1690.0,60,487,194,253,59,21,85,146,0.432,0.392,0.840
200782,P.J. Tucker,39.0,486.0,31,52,85,16,16,7,8,54,0.360,0.371,1.000
201142,Kevin Durant,35.0,2791.0,75,2032,495,378,69,91,244,133,0.523,0.413,0.856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641854,Craig Porter,24.0,649.0,51,285,109,118,21,15,45,40,0.509,0.353,0.732
1641871,Duop Reath,28.0,1214.0,68,617,250,67,33,38,52,161,0.461,0.359,0.742
1641926,Dexter Dennis,25.0,30.0,4,22,9,4,0,1,3,1,0.526,0.125,0.333


# --- 

## Add Per Game Stats 

In [5]:
pg_stats = ['PPG', 'RPG', 'APG', 'SPG', 'BPG', 'ToPG', 'PFPG', 'MPG']
prev_stats = ['PTS', 'REB',	'AST',	'STL',	'BLK',	'TOV',	'PF', 'MIN']
season_stats = add_pg_stats(season_stats, pg_stats, prev_stats) 
season_stats 

,Name,Age,MIN,GP,PTS,REB,AST,STL,BLK,TOV,PF,FG_PCT,FG3_PCT,FT_PCT,PPG,RPG,APG,SPG,BPG,ToPG,PFPG,MPG
ID,,,,,,,,,,,,,,,,,,,,,,
2544,LeBron James,39.0,2504.0,71,1822,518,589,89,38,245,78,0.540,0.410,0.750,25.7,7.3,8.3,1.3,0.5,3.5,1.1,35.3
101108,Chris Paul,39.0,1531.0,58,533,225,393,70,6,76,108,0.441,0.371,0.827,9.2,3.9,6.8,1.2,0.1,1.3,1.9,26.4
200768,Kyle Lowry,38.0,1690.0,60,487,194,253,59,21,85,146,0.432,0.392,0.840,8.1,3.2,4.2,1.0,0.4,1.4,2.4,28.2
200782,P.J. Tucker,39.0,486.0,31,52,85,16,16,7,8,54,0.360,0.371,1.000,1.7,2.7,0.5,0.5,0.2,0.3,1.7,15.7
201142,Kevin Durant,35.0,2791.0,75,2032,495,378,69,91,244,133,0.523,0.413,0.856,27.1,6.6,5.0,0.9,1.2,3.3,1.8,37.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641854,Craig Porter,24.0,649.0,51,285,109,118,21,15,45,40,0.509,0.353,0.732,5.6,2.1,2.3,0.4,0.3,0.9,0.8,12.7
1641871,Duop Reath,28.0,1214.0,68,617,250,67,33,38,52,161,0.461,0.359,0.742,9.1,3.7,1.0,0.5,0.6,0.8,2.4,17.9
1641926,Dexter Dennis,25.0,30.0,4,22,9,4,0,1,3,1,0.526,0.125,0.333,5.5,2.2,1.0,0.0,0.2,0.8,0.2,7.5


## Export to SQL Database 

In [6]:
db_path = 'sqlite:///../../DB/ballbase.db'
season = "2023_24" 
table_name = 'Season_Stats_' + season 

engine = create_engine(db_path, echo=False) 

season_stats.to_sql(table_name, con=engine, if_exists='replace', index=True) 

531

## Separate into Categories 

In [7]:
# Total Stats Table 
totals = season_stats.loc[:, 'Name':'FT_PCT'].sort_values('PTS', ascending=False) 

# Per Game Stats Table  
avgs = season_stats.loc[:,['Name', 'Age', 'PPG', 'RPG', 'APG', 'SPG', 'BPG', 'ToPG',	'PFPG', 'MPG', 'FG_PCT', 'FG3_PCT',	'FT_PCT']].sort_values('PPG', ascending=False)  

totals 
avgs 

contents = [totals, avgs]

## Convert to HTML 

In [8]:
# Create HTML table version 
for i in range(len(contents)):
    contents[i] = contents[i].to_html(table_id="table", classes = "table table-hover table-stripped table-bordered table-striped", index=False)

# Get current date 
from datetime import datetime 
curr_date = datetime.today().strftime('%Y-%m-%d %H:%M:%S') 
# Create page content 
for i in range(len(contents)):
    contents[i] = f"""
    {{%extends 'TableBase.html'%}}

    {{% block table %}}
    <p style="text-align:center;"> Last Update: {curr_date} ET</p>
    
    {contents[i]}
    {{% endblock %}}
"""

## Output to Page 

In [9]:
def write_to_file(path, content): 
    file = open(path, "w", encoding="utf-8") 
    file.write(content) 
    file.close 


file_names=["Totals.html", "PerGame.html"] 

for i in range(len(contents)): 
    path = f"../../templates/Tables/{file_names[i]}" 
    write_to_file(path, contents[i]) 
    
    